In [73]:
%load_ext autoreload
%autoreload 2
import pickle
from astropy.table import QTable
from astropy import units as u
import numpy as np


from nfw_profile_heating import NFWProfile
from statistics_functions import chi_squared

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
# import datasets
with open('data/mcxc_psz_combined.pkl', 'rb') as f:
    comb_data = pickle.load(f)
with open('data/REFLEX.p', 'rb') as g:
    errors_dict = pickle.load(g)

In [41]:
errors_dict


{'RXCJ0003.1-0605': 0.6179094419486882,
 'RXCJ0003.2-3555': 0.07247964293288481,
 'RXCJ0003.8+0203': 0.14703710320470523,
 'RXCJ0006.0-3443': 0.22761947095956642,
 'RXCJ0011.3-2851': 0.09295868677236488,
 'RXCJ0013.6-1930': 0.13354210223189258,
 'RXCJ0014.3-3023': 1.7293412740877763,
 'RXCJ0014.3-6604': 0.3376996999677323,
 'RXCJ0017.5-3509': 0.1323933419454966,
 'RXCJ0020.7-2542': 0.3062061422609284,
 'RXCJ0025.5-3302': 0.04639701540882626,
 'RXCJ0027.3-5015': 0.33035779817579547,
 'RXCJ0028.6-2338': 0.1762232582198019,
 'RXCJ0040.1-5607': 0.013688862424430896,
 'RXCJ0041.8-0918': 0.1641047451727569,
 'RXCJ0042.1-2832': 0.24537675727894,
 'RXCJ0043.4-2037': 0.8483302244986201,
 'RXCJ0049.4-2931': 0.21505671067355567,
 'RXCJ0052.7-8015': 0.6783457890981536,
 'RXCJ0056.3-0112': 0.07848718985966151,
 'RXCJ0057.8-6648': 0.2536801907396271,
 'RXCJ0102.7-2152': 0.0925488665939969,
 'RXCJ0105.5-2439': 0.6647784187183639,
 'RXCJ0106.8-0229': 0.5185702798856171,
 'RXCJ0107.8-3643': 0.148150670

In [36]:
keys = reflex_appended['OName'].tolist()
print(all(key in errors_dict for key in keys))
missing_keys = [key for key in keys if key not in errors_dict]
print(missing_keys)
print(len(missing_keys))
# could not find errors for 37 clusters

False
['RXCJ0034.6-0208', 'RXCJ0217.2-5244', 'RXCJ0220.9-3829', 'RXCJ0229.3-3332', 'RXCJ0232.2-4420', 'RXCJ0303.7-7752', 'RXCJ0331.1-2100', 'RXCJ0413.9-3805', 'RXCJ0437.1+0043', 'RXCJ0501.6+0110', 'RXCJ0510.7-0801', 'RXCJ0528.2-2942', 'RXCJ0532.9-3701', 'RXCJ0616.8-4748', 'RXCJ0631.3-5610', 'RXCJ0738.1-7506', 'RXCJ1013.5-1350', 'RXCJ1038.4-2454', 'RXCJ1115.8+0129', 'RXCJ1206.2-0848', 'RXCJ1215.4-3900', 'RXCJ1314.4-2515', 'RXCJ1317.1-3821', 'RXCJ1347.5-1144', 'RXCJ1504.1-0248', 'RXCJ1514.9-1523', 'RXCJ1558.3-1410', 'RXCJ1855.8-6654', 'RXCJ1947.3-7623', 'RXCJ2003.5-2323', 'RXCJ2014.8-2430', 'RXCJ2023.4-5535', 'RXCJ2030.7-3532', 'RXCJ2031.8-4037', 'RXCJ2125.2-0657', 'RXCJ2218.8-0258', 'RXCJ2344.2-0422']
37


In [46]:
reflex_data = comb_data[comb_data['Sub-Cat']=='REFLEX']
reflex_appended = reflex_data.copy()
reflex_appended['L_err'] = reflex_appended['OName'].apply(lambda x: errors_dict.get(x))
reflex_clean = reflex_appended[reflex_appended['L_err'].notna()]
reflex_clean

,MCXC,OName,AName,Ah Am RAs Ed Em Es,RAdeg DEdeg,GLON GLAT,z_x,Cat,Sub-Cat,Scale,...,E_MSZ,e_MSZ,RedMAPPer,ACT,SPT,ISE,AMIevi,O,Comment,L_err
1,J0003.1-0605,RXCJ0003.1-0605,A2697,0 3 11.8 - 6 5 10,0.799 -6.086,92.169 -66.033,0.2320,NORAS/REFLEX,REFLEX,3.698,...,0.368165,0.369144,RMJ000311.6-060530.1,,,-10,-1000.0,1,NaN,0.617909
2,J0003.8+0203,RXCJ0003.8+0203,A2700,0 3 50.6 + 2 3 48,0.961 2.063,99.610 -58.637,0.0924,NORAS/REFLEX,REFLEX,1.719,...,0.282160,0.301448,RMJ000349.7+020359.3,,,-10,-1000.0,0,NaN,0.147037
3,J0006.0-3443,RXCJ0006.0-3443,A2721,0 6 3.0 -34 43 27,1.513 -34.724,352.147 -77.668,0.1147,NORAS/REFLEX,REFLEX,2.080,...,0.248869,0.226974,,,,-10,-1000.0,1,NaN,0.227619
6,J0011.3-2851,RXCJ0011.3-2851,A2734,0 11 20.7 -28 51 18,2.836 -28.855,19.562 -80.986,0.0620,NORAS/REFLEX,REFLEX,1.195,...,0.182667,0.188038,,,,-10,-1000.0,1,NaN,0.092959
9,J0013.6-1930,RXCJ0013.6-1930,A0013,0 13 38.3 -19 30 8,3.409 -19.502,72.276 -78.456,0.0940,NORAS/REFLEX,REFLEX,1.746,...,0.233331,0.245311,,,,-10,-1000.0,1,NaN,0.133542
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543,J2347.7-2808,RXCJ2347.7-2808,A4038,23 47 43.2 -28 8 29,356.930 -28.141,25.139 -75.862,0.0300,NORAS/REFLEX,REFLEX,0.601,...,0.114748,0.107849,,,,-10,-1000.0,1,NaN,0.038488
546,J2351.6-2605,RXCJ2351.6-2605,A2667,23 51 40.7 -26 5 1,357.919 -26.084,34.017 -76.609,0.2264,NORAS/REFLEX,REFLEX,3.631,...,0.365168,0.382711,,,,-10,-1000.0,1,NaN,0.993468
547,J2354.2-1024,RXCJ2354.2-1024,A2670,23 54 13.4 -10 24 46,358.556 -10.413,81.334 -68.530,0.0765,NORAS/REFLEX,REFLEX,1.450,...,0.216913,0.225417,RMJ235413.7-102508.5,,,-10,-1000.0,1,NaN,0.151354
549,J2357.0-3445,RXCJ2357.0-3445,A4059,23 57 2.3 -34 45 38,359.260 -34.761,356.359 -76.082,0.0475,NORAS/REFLEX,REFLEX,0.931,...,0.163126,0.149009,,,,-10,-1000.0,1,NaN,0.091267


In [47]:
L_err_list = reflex_clean['L_err'].tolist()
L_err_list

[0.6179094419486882,
 0.14703710320470523,
 0.22761947095956642,
 0.09295868677236488,
 0.13354210223189258,
 0.3376996999677323,
 1.7293412740877763,
 0.3062061422609284,
 0.04639701540882626,
 0.33035779817579547,
 0.1762232582198019,
 0.1641047451727569,
 0.24537675727894,
 0.8483302244986201,
 0.07848718985966151,
 0.0925488665939969,
 0.6647784187183639,
 0.04290117582179534,
 0.014446791901477143,
 0.04427524236779236,
 0.5714405637181501,
 0.05943654764724139,
 0.5045482838885549,
 0.2867256713699702,
 0.4436884192992519,
 0.2681984406120019,
 0.5796597097222473,
 0.09100651536661837,
 0.46449603444193566,
 0.8790525964936597,
 0.1551347321172641,
 0.09000598818961687,
 0.14363824973439615,
 0.05694014240691957,
 0.10325761146446225,
 0.06122966248764443,
 0.5496809384801187,
 0.18113312845470247,
 0.0935051826076239,
 0.06905150366042391,
 0.0946826534929151,
 0.058054929209745254,
 0.06957019099946796,
 0.06425621151153069,
 0.09946722237715844,
 0.08716212177329971,
 0.292404

In [50]:
def generate_nfws_from_combined_dataset(dataset, nrows=None):
    cls_data={'M500':dataset['MSZ'],
          'L500':dataset['L500'],
          'z':dataset['z_x']
         }
    units={
        'M500': 1e14*u.Msun,
        'L500': 1e37*u.W,
        'z': None
    }

    cls_table=QTable(cls_data, units=units)
    n=nrows or len(cls_table)
    
    return [
        NFWProfile(
            cls_table['z'][i],
            M500=cls_table['M500'][i],
            L500=cls_table['L500'][i]
        )
        for i in range(n)
    ]

In [62]:
nfws = generate_nfws_from_combined_dataset(reflex_clean)
L_obs = [nfw.L500.to(u.erg/u.s) for nfw in nfws]

In [63]:
L_obs

[<Quantity 6.107094e+44 erg / s>,
 <Quantity 8.46881e+43 erg / s>,
 <Quantity 1.809494e+44 erg / s>,
 <Quantity 1.086002e+44 erg / s>,
 <Quantity 1.236332e+44 erg / s>,
 <Quantity 2.827248e+44 erg / s>,
 <Quantity 1.1818114e+45 erg / s>,
 <Quantity 2.872451e+44 erg / s>,
 <Quantity 4.9467e+43 erg / s>,
 <Quantity 2.164603e+44 erg / s>,
 <Quantity 1.404549e+44 erg / s>,
 <Quantity 5.100085e+44 erg / s>,
 <Quantity 2.734146e+44 erg / s>,
 <Quantity 6.907787e+44 erg / s>,
 <Quantity 1.437223e+44 erg / s>,
 <Quantity 1.460216e+44 erg / s>,
 <Quantity 5.160525e+44 erg / s>,
 <Quantity 4.91051e+43 erg / s>,
 <Quantity 1.81484e+43 erg / s>,
 <Quantity 4.67516e+43 erg / s>,
 <Quantity 4.979596e+44 erg / s>,
 <Quantity 7.63259e+43 erg / s>,
 <Quantity 5.701343e+44 erg / s>,
 <Quantity 2.010759e+44 erg / s>,
 <Quantity 4.237381e+44 erg / s>,
 <Quantity 1.768408e+44 erg / s>,
 <Quantity 6.076038e+44 erg / s>,
 <Quantity 6.69697e+43 erg / s>,
 <Quantity 3.639374e+44 erg / s>,
 <Quantity 6.258801e+

In [75]:
variances = (np.array(L_err_list)*1e37*u.W).to(1e44*u.erg/u.s)

In [76]:
len(nfws), len(L_err_list)

(188, 188)

In [67]:
L_err_list

[0.6179094419486882,
 0.14703710320470523,
 0.22761947095956642,
 0.09295868677236488,
 0.13354210223189258,
 0.3376996999677323,
 1.7293412740877763,
 0.3062061422609284,
 0.04639701540882626,
 0.33035779817579547,
 0.1762232582198019,
 0.1641047451727569,
 0.24537675727894,
 0.8483302244986201,
 0.07848718985966151,
 0.0925488665939969,
 0.6647784187183639,
 0.04290117582179534,
 0.014446791901477143,
 0.04427524236779236,
 0.5714405637181501,
 0.05943654764724139,
 0.5045482838885549,
 0.2867256713699702,
 0.4436884192992519,
 0.2681984406120019,
 0.5796597097222473,
 0.09100651536661837,
 0.46449603444193566,
 0.8790525964936597,
 0.1551347321172641,
 0.09000598818961687,
 0.14363824973439615,
 0.05694014240691957,
 0.10325761146446225,
 0.06122966248764443,
 0.5496809384801187,
 0.18113312845470247,
 0.0935051826076239,
 0.06905150366042391,
 0.0946826534929151,
 0.058054929209745254,
 0.06957019099946796,
 0.06425621151153069,
 0.09946722237715844,
 0.08716212177329971,
 0.292404

In [85]:
def log_lik(p0, obs, variances, nfws):
    log_s0 = p0
    s0 = 10**log_s0*u.cm**2
    mx = 1e-2*u.GeV
    # prior
    if log_s0>0:
        return -np.inf
    # calculate model predicted L500
    predicted = [nfw.solve_for_L500(s0, mx) for nfw in nfws]
    X2 = chi_squared(predicted, obs, variances)
    return -X2/2

In [81]:
nfws_subset = nfws[:10]
vars_subset = variances[:10]
obs_subset = L_obs[:10]

In [90]:
log_lik(-25, obs_subset, vars_subset, nfws_subset)

<Quantity -4.1911408e+09>

In [93]:
log_lik(-20, obs_subset, vars_subset, nfws_subset)

<Quantity -2.34888436e+35>

In [92]:
log_lik(-30, obs_subset, vars_subset, nfws_subset)

<Quantity -1611737.0052506>